In [1]:
using Pkg; Pkg.activate("/home/dhairyagandhi96/modelzoo/model-zoo/script/../text/lang-detection"); Pkg.instantiate(); Pkg.add(PackageSpec(name="CUDAdrv", rev = "master")); Pkg.add(PackageSpec(name="CUDAnative", rev = "master")); Pkg.add(PackageSpec(name="CuArrays", rev = "master")); Pkg.add(PackageSpec(name="Flux", rev = "master"))

using Flux
using Flux: onehot, onehotbatch, crossentropy, reset!, throttle
using Unicode
using Random     # for 'shuffle'
using Statistics # for 'mean'

include("scrape.jl")
corpora = Dict()

cd(@__DIR__)
for file in readdir("corpus")
  lang = Symbol(match(r"(.*)\.txt", file).captures[1])
  corpus = split(String(read("corpus/$file")), ".")
  corpus = strip.(Unicode.normalize.(corpus, casefold=true, stripmark=true))
  corpus = filter(!isempty, corpus)
  corpora[lang] = corpus
end

langs = collect(keys(corpora))
alphabet = ['a':'z'; '0':'9'; ' '; '\n'; '_']

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/JuliaGPU/CUDAdrv.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/text/lang-detection/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/text/lang-detection/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CUDAnative.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/text/lang-detection/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/text/lang-detection/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CuArrays.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/text/lang-detection/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/text/lang-detection/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/FluxML/Flux.jl.git`
 Resolving package versions...
  Upda

39-element Array{Char,1}:
 'a' 
 'b' 
 'c' 
 'd' 
 'e' 
 'f' 
 'g' 
 'h' 
 'i' 
 'j' 
 ⋮   
 '4' 
 '5' 
 '6' 
 '7' 
 '8' 
 '9' 
 ' ' 
 '\n'
 '_' 

See which chars will be represented as "unknown"

In [2]:
unique(filter(x -> x ∉ alphabet, join(vcat(values(corpora)...))))

dataset = [(onehotbatch(s, alphabet, '_'), onehot(l, langs))
           for l in langs for s in corpora[l]] |> shuffle

train, test = dataset[1:end-100], dataset[end-99:end]

N = 15

scanner = Chain(Dense(length(alphabet), N, σ), LSTM(N, N))
encoder = Dense(N, length(langs))

function model(x)
  state = scanner.(x.data)[end]
  reset!(scanner)
  softmax(encoder(state))
end

loss(x, y) = crossentropy(model(x), y)

testloss() = mean(loss(t...) for t in test)

opt = ADAM(params(scanner, encoder))
evalcb = () -> @show testloss()

Flux.train!(loss, train, opt, cb = throttle(evalcb, 10))

┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Warning: train!(loss, data, opt) is deprecated; use train!(loss, params, data, opt) instead
│   caller = ip:0x0
└ @ Core :-1
testloss() = 1.7248776f0 (tracked)
testloss() = 1.5436935f0 (tracked)
testloss() = 1.5439239f0 (tracked)
testloss() = 1.5181676f0 (tracked)
testloss() = 1.5167382f0 (tracked)
testloss() = 1.501078f0 (tracked)
testloss() = 1.4949843f0 (tracked)
testloss() = 1.4911517f0 (tracked)
testloss() = 1.4671066f0 (tracked)
testloss() = 1.4569464f0 (tracked)
testloss() = 1.44892f0 (tracked)
testloss() = 1.4186455f0 (tracked)
testloss() = 1.3899051f0 (tracked)
testloss() = 1.2792935f0 (tracked)
testloss() = 1.4331985f0 (tracked)
testloss() = 1.3875095f0 (tracked)
testloss() = 1.210101f0 (tracked)
testloss() = 1.1922368f0 (tracked)
testloss() = 1.1642448f0 (tracked)
testloss() = 1.099103f0 (tracked)
testloss() = 1.4442871f0 (tracked)
testloss() = 1.15903